In this notebook I experiment with creating obj files from the heat flux data (a small toy subset array from the real data).  
For now, each time point would have its obj file.

#### obj file format:
```
# List of geometric vertices, with (x, y, z, [w]) coordinates. (z is height)
# w is optional and defaults to 1 - it's the weight required for rational curves and surfaces.
# Non-rational curve - When a curve's control points all have the same weight.
# Some applications support vertex colors, by putting red, green and blue values after x y and z (ranging 0-1), instead of w.
v 0.123 0.234 0.345 1.0

# Optional:
# List of texture coordinates, in (u, [v, w]) coordinates, these will vary between 0 and 1. 
# v, w are optional and default to 0.
vt 0.500 1 [0]

# Optional:
# List of vertex normals in (x,y,z) form; normals might not be unit vectors.
# Many times these aren't used, because the 'f' face command will use the order the 'v' commands are given to determine the normal instead.
vn 0.707 0.000 0.707

# Optional
# Parameter space vertices in (u, [v, w]) form; free form geometry statement (????)
vp 0.310000 3.210000 2.100000

# Polygonal face element:
f 1 2 3                # v1, v2, v3 make a face
f 3/1 4/2 5/3          # v3+vt1, v4+vt2 ... make a face
f 6/4/1 3/5/3 7/6/5    # v6+vt4+vn1, ... make a face
f 7//1 8//2 9//3       # v7+vn1, ... make a face

# Reference materials: 
# The use material command lets you name a material to use. 
# All 'f' face commands that follow will use the same material, until another usemtl command is encountered.
usemtl name

# Obj files support higher-order surfaces using several different kinds of interpolation, such as Taylor and B-splines, although support for those features in third party file readers is far from universal.

```

Sources:  
https://www.cs.cmu.edu/~mbz/personal/graphics/obj.html  
https://en.wikipedia.org/wiki/Wavefront_.obj_file

In [1]:
import numpy as np
from datetime import date
from os import path, makedirs

#### Import original toy array

In [2]:
org_path = 'UFZ_RemoteSensing/sliced_heat_flux_array_as_toy.npy'

In [3]:
org_array = np.load(org_path)

In [4]:
org_array.shape

(10, 601, 1233)

Those dims are time, longitude, latitude

In [5]:
### setting the invalid value:
invalid_val = np.min(org_array)
invalid_val

-9999999.0

In [6]:
### Create otput dir:
today = date.today().strftime("%y%m%d")
output_dir = f'{today}_toy_obj_files'

makedirs(output_dir)#, exist_ok=True)

### Creating an obj file
For now just with `v` and `f` lines.

#### Creating vertices & faces:

In [40]:
def get_usable_Vs(mat, mat_size):
    # Init bool array - which vertices particiate in a face: 
    bool_mat = np.full(mat_size, False)
    
    for i in range(1,mat_size[0]):
        for j in range(1,mat_size[1]-1):
            
            if (mat[i,j]!=invalid_val) and (mat[i-1,j]!=invalid_val):
                ## each vertex can create two faces (if it and neighbors are valid)
                ## first face participating vertices:
                if mat[i-1,j-1]!=invalid_val:
                    bool_mat[i,j], bool_mat[i-1,j], bool_mat[i-1,j-1] = True, True, True
                # second face:
                if mat[i,j+1]!=invalid_val:
                    bool_mat[i,j], bool_mat[i-1,j], bool_mat[i,j+1] = True, True, True
                
    return bool_mat

In [46]:
# This function creates the vertices lines in the .obj file.
# line format / example:
# v x y z / v 5 6 25 

def add_v_in_obj(i, j, val):
    return f'v {i} {j} {val}'

In [47]:
## For now, for simplicity, vertices missing a neighbor dont create faces
# (due to empty value -9999999 or array boarders)

def add_f_to_obj(v_idx, v_idx_neigh1, v_idx_neigh2):
    return f'f {v_idx} {v_idx_neigh1} {v_idx_neigh2}'
    

In [52]:
## multi_index order (e.g. in arr.shape=(2,3)):
## v0=<(0,0)> v1=<(0,1)> v2=<(0,2)> v3=<(1,0)> v4=<(1,1)> v5=<(1, 2)>
## But this is only when all vertices are used! 
## Need a more complex approach to track vertices idxs.

mat_size = org_array.shape[1:]

# Time iter:
for i_mat,mat in enumerate(org_array):
    
    ## Commented as currently makes more sense to do it with two loops
    ## Creating an iterator:
    # it = np.nditer(mat, flags=['multi_index'])
    # for x in it:
    #     if x>-999999:
    #         make_v_line(it.multi_index, x)
    
    
    # Check which vertices will actually be used in faces:
    is_v_mat = get_usable_Vs(mat, mat_size)
    
    v_idx = 0
    
    ## make lists of - vertices lines and faces lines (in obj format)
    Vs_obj = []
    Fs_obj = []
    
    v_idxs_curr_row = []
    
    for i in range(mat_size[0]):
        for j in range(mat_size[1]):
            
            ## Make vertex:
            if is_v_mat[i,j]:
                Vs_obj.append(add_v_in_obj(i, j, mat[i,j]))
                
                ## Make face: 
                # First row doesn't have neighbors to create faces with.
                if (i!=0) and (j!=0) and (j!=mat_size[1]-1):
                    ## For each vertex (up to) two faces are created:
                    # (V i j , V i-1 j-1, V i-1 j) & (V i j , V i-1 j, V i j+1)
                    
                    # excluding face if neighbor is invalid.
                    
                    ## First face: 
                    if is_v_mat[i-1,j] and is_v_mat[i-1,j-1]:
                        Fs_obj.append(add_f_to_obj(v_idx, v_idxs_prev_row[j], v_idxs_prev_row[j-1]))
                    # Second face:
                    if is_v_mat[i-1,j] and is_v_mat[i,j+1]:
                        Fs_obj.append(add_f_to_obj(v_idx, v_idxs_prev_row[j], v_idx+1))
                
                v_idxs_curr_row.append(v_idx)
                v_idx += 1
                
            else:
                v_idxs_curr_row.append(False)

        v_idxs_prev_row, v_idxs_curr_row = v_idxs_curr_row, []
                
    ## create obj file for current time point:
    with open(path.join(output_dir, f't{i_mat}.obj'), 'w') as f:
        f.write('\n'.join(Vs_obj))
    with open(path.join(output_dir, f't{i_mat}.obj'), 'a') as f:
        f.write('\n'.join(Fs_obj))
        


In [53]:
## Test if i get less vertices after the change in code to use only needed vertices:

with open(path.join(output_dir, 't0.obj'), 'r') as f:
    lines = f.read().split('\n')
    
with open(path.join('221214_toy_obj_files_pre_neighbor_check','t0.obj')) as f:
    lines1 = f.read().split('\n')
    
    

In [54]:
len(lines), len(lines1)
### Oh, its because im using the last line here too..

(1334335, 1333583)

In [55]:
Vs_line = [l for l in lines if 'v' in l]
Vs_line1 = [l for l in lines1 if 'v' in l]

In [56]:
len(Vs_line), len(Vs_line1)
## Ok, so i do have less vertices, but more faces.

(450621, 450673)